# Анализ факторов успеваемости студентов

## Введение

Данный ноутбук содержит комплексный анализ факторов, влияющих на успеваемость студентов. Мы проведем:
- Загрузку и первичный анализ данных
- Статистический анализ
- Построение и оценку предсказательной модели
- Визуализацию результатов

## Настройка окружения

Сначала импортируем необходимые библиотеки и настроим параметры отображения.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Настройка отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Настройка стиля графиков
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False

## 1. Загрузка и первичный анализ данных

### 1.1 Загрузка данных

Загружаем датасет и проводим первичный осмотр данных. Это важный этап для понимания структуры и качества данных.

In [ ]:
def load_data(file_path):
    """Загрузка данных из CSV файла с базовой проверкой"""
    try:
        data = pd.read_csv(file_path, encoding='utf-8')
        print("Данные успешно загружены")
        return data
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return None

data = load_data('data/StudentPerformanceFactors.csv')
data.head()

### 1.2 Исследовательский анализ данных (EDA)

Проведем базовый анализ данных для понимания их структуры и распределения.

In [ ]:
# Базовая информация о датасете
print("\nОбщая информация о датасете:")
print(data.info())

print("\nСтатистическое описание числовых признаков:")
print(data.describe())

print("\nКоличество пропущенных значений:")
print(data.isnull().sum())

## 2. Визуализация данных

### 2.1 Распределение целевой переменной

Визуализируем распределение оценок студентов для понимания общей картины успеваемости.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['Exam_Score'], kde=True)
plt.title('Распределение итоговых оценок')
plt.xlabel('Оценка')
plt.ylabel('Частота')
plt.show()

### 2.2 Корреляционный анализ

Построим тепловую карту корреляций для выявления взаимосвязей между признаками.

In [ ]:
numeric_data = data.select_dtypes(include=[np.number])
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Корреляционная матрица')
plt.tight_layout()
plt.show()

## 3. Предобработка данных

Подготовим данные для моделирования, включая:
- Обработку пропущенных значений
- Кодирование категориальных переменных
- Стандартизацию числовых признаков

In [ ]:
def preprocess_data(data):
    """Комплексная предобработка данных"""
    df = data.copy()
    
    # One-hot encoding для категориальных переменных
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
    
    # Стандартизация числовых признаков
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    numeric_features = [col for col in numeric_columns if col != 'Exam_Score']
    
    scaler = StandardScaler()
    df[numeric_features] = scaler.fit_transform(df[numeric_features])
    
    return df, scaler

data_processed, scaler = preprocess_data(data)

## 4. Моделирование

### 4.1 Подготовка данных для обучения

Разделим данные на обучающую, валидационную и тестовую выборки.

In [ ]:
X = data_processed.drop('Exam_Score', axis=1)
y = data_processed['Exam_Score']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print("Размеры выборок:")
print(f"Обучающая выборка: {X_train.shape}")
print(f"Валидационная выборка: {X_val.shape}")
print(f"Тестовая выборка: {X_test.shape}")

### 4.2 Подбор гиперпараметров

Используем GridSearchCV для поиска оптимальных параметров модели.

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5, 6],
    'n_estimators': [100, 200],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print("Лучшие параметры:", grid_search.best_params_)

## 5. Оценка модели

Проведем комплексную оценку качества модели на всех выборках.

In [ ]:
def evaluate_model(model, X, y, dataset_name):
    """Оценка качества модели на заданной выборке"""
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    
    print(f"\nРезультаты на {dataset_name} выборке:")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
    print(f"MAE: {mae:.4f}")

evaluate_model(best_model, X_train, y_train, "обучающей")
evaluate_model(best_model, X_val, y_val, "валидационной")
evaluate_model(best_model, X_test, y_test, "тестовой")

## 6. Анализ важности признаков

Визуализируем важность различных факторов для предсказания успеваемости.

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(10))
plt.title('Топ-10 важных факторов')
plt.tight_layout()
plt.show()

## 7. Выводы и рекомендации

На основе проведенного анализа можно сделать следующие выводы:

1. Наиболее важные факторы успеваемости:
   - Посещаемость занятий
   - Количество часов самостоятельной подготовки
   - Предыдущие академические результаты

2. Демографические факторы оказывают минимальное влияние

3. Рекомендации по улучшению успеваемости:
   - Фокус на повышение посещаемости занятий
   - Организация эффективного учебного времени
   - Обеспечение доступа к образовательным ресурсам
   - Вовлечение родителей в образовательный процесс

## 8. Визуализация предсказаний

Проанализируем качество предсказаний модели с помощью различных визуализаций.

In [ ]:
def visualize_predictions(model, X_test, y_test):
    """Визуализация предсказаний модели"""
    # Получаем предсказания
    y_pred = model.predict(X_test)
    
    # График сравнения реальных и предсказанных значений
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Реальные значения')
    plt.ylabel('Предсказанные значения')
    plt.title('Сравнение реальных и предсказанных значений')
    plt.show()
    
    # График остатков
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True)
    plt.xlabel('Остатки')
    plt.ylabel('Частота')
    plt.title('Распределение остатков')
    plt.show()
    
    return residuals

residuals = visualize_predictions(best_model, X_test, y_test)

## 9. Анализ ошибок модели

Проанализируем, в каких случаях модель делает наибольшие ошибки.

In [ ]:
def analyze_errors(X_test, y_test, y_pred, threshold=2):
    """Анализ случаев с большими ошибками предсказания"""
    errors = np.abs(y_test - y_pred)
    large_errors = errors > threshold * errors.std()
    
    error_analysis = pd.DataFrame({
        'actual': y_test[large_errors],
        'predicted': y_pred[large_errors],
        'error': errors[large_errors]
    })
    
    print(f"Количество больших ошибок (>{threshold} std): {large_errors.sum()}")
    print(f"Процент больших ошибок: {100 * large_errors.sum() / len(y_test):.2f}%")
    
    return error_analysis

y_pred = best_model.predict(X_test)
error_analysis = analyze_errors(X_test, y_test, y_pred)

## 10. Улучшение модели

Рассмотрим возможные способы улучшения модели:

1. Инженерия признаков:
   - Создание взаимодействий между важными признаками
   - Нелинейные преобразования числовых признаков
   - Агрегация связанных характеристик

2. Ансамблевые методы:
   - Стекинг нескольких моделей
   - Бэггинг для уменьшения переобучения
   - Бустинг для улучшения точности

In [ ]:
def create_interaction_features(X):
    """Создание признаков взаимодействия"""
    X = X.copy()
    
    # Добавляем взаимодействия между важными признаками
    X['attendance_study'] = X['Attendance'] * X['Hours_Studied']
    X['attendance_sleep'] = X['Attendance'] * X['Sleep_Hours']
    X['study_sleep'] = X['Hours_Studied'] * X['Sleep_Hours']
    
    # Нелинейные преобразования
    X['hours_studied_squared'] = X['Hours_Studied'] ** 2
    X['attendance_squared'] = X['Attendance'] ** 2
    
    return X

# Создаем новые признаки
X_train_enhanced = create_interaction_features(X_train)
X_test_enhanced = create_interaction_features(X_test)

## 11. Сохранение модели

Сохраним лучшую модель и параметры предобработки для дальнейшего использования.

In [ ]:
import joblib

# Создаем директорию для моделей, если её нет
if not os.path.exists('models'):
    os.makedirs('models')

# Сохраняем модель и скейлер
joblib.dump(best_model, 'models/student_performance_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')

print("Модель и параметры предобработки сохранены в директории 'models'")

## 12. Заключение

В этом анализе мы:
1. Провели комплексный анализ факторов успеваемости студентов
2. Построили и оценили предсказательную модель
3. Выявили ключевые факторы успеха
4. Предложили рекомендации по улучшению успеваемости

### Дальнейшие шаги:
1. Сбор дополнительных данных о студентах
2. Эксперименты с другими алгоритмами машинного обучения
3. Разработка системы раннего предупреждения для студентов группы риска
4. Создание интерактивного дашборда для мониторинга успеваемости